In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd

from data_preprocessing.spatio_temporal_features import SpatioTemporalFeatures
from utils.common import GroupsInfo, get_mapping_cols_tobii, get_mapping_cols, get_mapping_cols_centroids
from config.constants import TRAJECTORY_SAMPLES_PATH

In [2]:
TRAJECTORY_SAMPLES_PATH = "/home/tmr/Documents/PhD/My_PhD/code/magni-dash/data/short"

In [3]:
files = os.listdir(os.path.join(TRAJECTORY_SAMPLES_PATH, "Scenario2"))
files_target = list(filter(lambda x: x.endswith(".csv"), files))
files_target

['Shortened_170522_SC2_R2.csv']

In [4]:
FILE_IDX = 0

In [5]:
files_target[FILE_IDX]

'Shortened_170522_SC2_R2.csv'

In [6]:
sync_df = pd.read_csv(os.path.join(TRAJECTORY_SAMPLES_PATH, "Scenario2", files_target[FILE_IDX]), index_col="Frame")

In [7]:
sync_df[sync_df.columns[sync_df.columns.str.contains("Helmet_10 Centroid")]].isna().sum() / sync_df.shape[0]

Helmet_10 Centroid_X    0.0
Helmet_10 Centroid_Y    0.0
Helmet_10 Centroid_Z    0.0
dtype: float64

In [8]:
for m in range(1,5):
    print("X", sync_df[sync_df.columns[sync_df.columns.str.contains(f"Helmet_10 - {m} X")]].isna().sum() / sync_df.shape[0])
    print("Y", sync_df[sync_df.columns[sync_df.columns.str.contains(f"Helmet_10 - {m} Y")]].isna().sum() / sync_df.shape[0])
    print("Z", sync_df[sync_df.columns[sync_df.columns.str.contains(f"Helmet_10 - {m} Z")]].isna().sum() / sync_df.shape[0])

X Helmet_10 - 1 X    0.0
dtype: float64
Y Helmet_10 - 1 Y    0.0
dtype: float64
Z Helmet_10 - 1 Z    0.0
dtype: float64
X Helmet_10 - 2 X    0.0
dtype: float64
Y Helmet_10 - 2 Y    0.0
dtype: float64
Z Helmet_10 - 2 Z    0.0
dtype: float64
X Helmet_10 - 3 X    0.0
dtype: float64
Y Helmet_10 - 3 Y    0.0
dtype: float64
Z Helmet_10 - 3 Z    0.0
dtype: float64
X Helmet_10 - 4 X    0.0
dtype: float64
Y Helmet_10 - 4 Y    0.0
dtype: float64
Z Helmet_10 - 4 Z    0.0
dtype: float64


In [9]:
sync_df[sync_df.columns[sync_df.columns.str.contains("G3D")]].isna().sum() / sync_df.shape[0]

Helmet_5 TB2_G3D_X    0.005988
Helmet_5 TB2_G3D_Y    0.005988
Helmet_5 TB2_G3D_Z    0.005988
Helmet_6 TB3_G3D_X    1.000000
Helmet_6 TB3_G3D_Y    1.000000
Helmet_6 TB3_G3D_Z    1.000000
dtype: float64

In [10]:
def get_best_markers(input_df: pd.DataFrame):
    """Get markers with lowest amount of NaN values"""
    x_coordinate = input_df[input_df.columns[input_df.columns.str.endswith("X")]]
    x_cols = x_coordinate.columns

    instances = set(x_coordinate.columns.str.split(" - ").str[0])
    instances = list(filter(lambda x: len(x.split(" ")) == 1, instances))
    nan_counter_by_marker = {}
    for instance_id in instances:
        nan_counter_by_marker[instance_id] = {}
        markers = (
            x_coordinate[x_cols[x_cols.str.startswith(f"{instance_id} -")]]
            .columns.str.split(regex=r" (/d) ")
            .str[2]
        )
        for marker_id in markers:
            n_nans = (
                x_coordinate[f"{instance_id} - {marker_id} X"]
                .isna()
                .sum()
            )
            nan_counter_by_marker[instance_id][marker_id] = n_nans
    print(nan_counter_by_marker)
    return nan_counter_by_marker

In [11]:
best_markers = get_best_markers(sync_df)

{'Helmet_1': {'1': 158, '2': 0, '3': 48}, 'Helmet_2': {'1': 0, '2': 16, '3': 17, '4': 11}, 'Helmet_5': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0}, 'LO1': {'1': 167, '2': 167, '3': 167, '4': 167, '5': 167, '6': 167, '7': 167, '8': 167}, 'Helmet_4': {'1': 92, '2': 158, '3': 130, '4': 167, '5': 161}, 'DARKO_Robot': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0}, 'Helmet_8': {'1': 0, '2': 0, '3': 5, '4': 0, '5': 0}, 'Helmet_10': {'1': 0, '2': 0, '3': 0, '4': 0}, 'Helmet_6': {'1': 0, '2': 163, '3': 144, '4': 167, '5': 72}}


In [12]:
def preprocess_df(raw_df: pd.DataFrame) -> pd.DataFrame:
    """interpolation and divide by 1000 to get measurements in meters"""
    preprocessed_df = raw_df.copy()
    trajectories_condition = preprocessed_df.columns[
            (preprocessed_df.columns.str.endswith(" X"))
            | ((preprocessed_df.columns.str.endswith(" Y")))
            | ((preprocessed_df.columns.str.endswith(" Z")))
        ]
    preprocessed_df[trajectories_condition] /= 1000
    return preprocessed_df

In [13]:
sync_df[sync_df.columns[sync_df.columns.str.endswith(" X")]].columns

Index(['DARKO_Robot - 1 X', 'DARKO_Robot - 2 X', 'DARKO_Robot - 3 X',
       'DARKO_Robot - 4 X', 'DARKO_Robot - 5 X', 'DARKO_Robot - 6 X',
       'DARKO_Robot - 7 X', 'Helmet_1 - 1 X', 'Helmet_1 - 2 X',
       'Helmet_1 - 3 X', 'Helmet_10 - 1 X', 'Helmet_10 - 2 X',
       'Helmet_10 - 3 X', 'Helmet_10 - 4 X', 'Helmet_2 - 1 X',
       'Helmet_2 - 2 X', 'Helmet_2 - 3 X', 'Helmet_2 - 4 X', 'Helmet_4 - 1 X',
       'Helmet_4 - 2 X', 'Helmet_4 - 3 X', 'Helmet_4 - 4 X', 'Helmet_4 - 5 X',
       'Helmet_5 - 1 X', 'Helmet_5 - 2 X', 'Helmet_5 - 3 X', 'Helmet_5 - 4 X',
       'Helmet_5 - 5 X', 'Helmet_6 - 1 X', 'Helmet_6 - 2 X', 'Helmet_6 - 3 X',
       'Helmet_6 - 4 X', 'Helmet_6 - 5 X', 'Helmet_8 - 1 X', 'Helmet_8 - 2 X',
       'Helmet_8 - 3 X', 'Helmet_8 - 4 X', 'Helmet_8 - 5 X', 'LO1 - 1 X',
       'LO1 - 2 X', 'LO1 - 3 X', 'LO1 - 4 X', 'LO1 - 5 X', 'LO1 - 6 X',
       'LO1 - 7 X', 'LO1 - 8 X'],
      dtype='object')

In [14]:
preprocessed_trajectories = preprocess_df(sync_df)

In [15]:
preprocessed_trajectories[
    preprocessed_trajectories.columns[
        preprocessed_trajectories.columns.str.contains("G3D")
    ]
].isna().sum() / preprocessed_trajectories.shape[0]

Helmet_5 TB2_G3D_X    0.005988
Helmet_5 TB2_G3D_Y    0.005988
Helmet_5 TB2_G3D_Z    0.005988
Helmet_6 TB3_G3D_X    1.000000
Helmet_6 TB3_G3D_Y    1.000000
Helmet_6 TB3_G3D_Z    1.000000
dtype: float64

In [16]:
moving_agents = preprocessed_trajectories.columns[
    (preprocessed_trajectories.columns.str.startswith("Helmet"))
    | (preprocessed_trajectories.columns.str.startswith("LO1"))
].tolist()
moving_agents_labels = set(map(lambda x: x.split(" - ")[0], moving_agents))
moving_agents_labels = list(filter(lambda x: len(x.split(" ")) == 1, moving_agents_labels))

In [17]:
def extract_features(
    out_df: pd.DataFrame,
    magents_labels,
    darko_label,
) -> pd.DataFrame:
    """Extract features to be used in the profiles section such as speed

    Parameters
    ----------
    input_df
        raw pandas DataFrame
    magents_labels
        moving agents labels
    darko_label
        darko robot label

    Returns
    -------
        pandas DataFrame with the respective features computed
    """
    magents_labels = (
        [magents_labels] if isinstance(magents_labels, str) else magents_labels
    )
    elements_labels = magents_labels + [darko_label] if darko_label else magents_labels

    out_df = SpatioTemporalFeatures.get_speed(
        out_df,
        time_col_name="Time",
        element_name=elements_labels,
    )

    out_df = out_df.reset_index()
    return out_df

In [18]:
moving_agents_cols = preprocessed_trajectories.columns[(preprocessed_trajectories.columns.str.endswith(" X"))
            | ((preprocessed_trajectories.columns.str.endswith(" Y")))
            | ((preprocessed_trajectories.columns.str.endswith(" Z")))]

In [19]:
moving_agents_cols

Index(['DARKO_Robot - 1 X', 'DARKO_Robot - 1 Y', 'DARKO_Robot - 1 Z',
       'DARKO_Robot - 2 X', 'DARKO_Robot - 2 Y', 'DARKO_Robot - 2 Z',
       'DARKO_Robot - 3 X', 'DARKO_Robot - 3 Y', 'DARKO_Robot - 3 Z',
       'DARKO_Robot - 4 X',
       ...
       'LO1 - 5 Z', 'LO1 - 6 X', 'LO1 - 6 Y', 'LO1 - 6 Z', 'LO1 - 7 X',
       'LO1 - 7 Y', 'LO1 - 7 Z', 'LO1 - 8 X', 'LO1 - 8 Y', 'LO1 - 8 Z'],
      dtype='object', length=138)

In [20]:
moving_agents_labels

['Helmet_4',
 'LO1',
 'Helmet_8',
 'Helmet_6',
 'Helmet_2',
 'Helmet_10',
 'Helmet_1',
 'Helmet_5']

In [21]:
features_df = extract_features(
        preprocessed_trajectories[["Time"] + moving_agents_cols.tolist()].copy(),
        magents_labels=moving_agents_labels,
        darko_label="DARKO_Robot",
    )

2023-05-25 16:35:30.225 INFO    data_preprocessing.spatio_temporal_features: {'Helmet_2 - 1 X_delta', 'DARKO_Robot - 2 Z_delta', 'Helmet_1 - 1 Y_delta', 'DARKO_Robot - 6 X_delta', 'LO1 - 7 Z_delta', 'Helmet_6 - 4 Y_delta', 'Helmet_2 - 3 X_delta', 'LO1 - 7 X_delta', 'Helmet_4 - 5 Z_delta', 'DARKO_Robot - 3 Y_delta', 'DARKO_Robot - 3 Z_delta', 'Helmet_5 - 1 Z_delta', 'Helmet_4 - 1 X_delta', 'Helmet_6 - 1 X_delta', 'LO1 - 8 Z_delta', 'Helmet_10 - 3 Z_delta', 'LO1 - 6 Z_delta', 'LO1 - 6 Y_delta', 'LO1 - 1 X_delta', 'DARKO_Robot - 7 X_delta', 'Helmet_4 - 2 Y_delta', 'Helmet_6 - 3 Z_delta', 'LO1 - 4 Z_delta', 'DARKO_Robot - 6 Z_delta', 'DARKO_Robot - 4 Z_delta', 'Helmet_10 - 4 Z_delta', 'LO1 - 4 Y_delta', 'Helmet_10 - 2 Z_delta', 'Helmet_10 - 4 X_delta', 'DARKO_Robot - 4 Y_delta', 'Helmet_10 - 3 Y_delta', 'Helmet_10 - 2 Y_delta', 'Helmet_1 - 1 X_delta', 'Helmet_8 - 1 Y_delta', 'Helmet_2 - 3 Y_delta', 'DARKO_Robot - 1 Y_delta', 'LO1 - 4 X_delta', 'Helmet_2 - 4 X_delta', 'Helmet_10 - 4 Y_delta

In [22]:
features_df[features_df.columns[features_df.columns.str.endswith("speed (m/s)")]]

,Helmet_4 - 1 speed (m/s),Helmet_4 - 2 speed (m/s),Helmet_4 - 3 speed (m/s),Helmet_4 - 4 speed (m/s),Helmet_4 - 5 speed (m/s),LO1 - 1 speed (m/s),LO1 - 2 speed (m/s),LO1 - 3 speed (m/s),LO1 - 4 speed (m/s),LO1 - 5 speed (m/s),...,Helmet_5 - 3 speed (m/s),Helmet_5 - 4 speed (m/s),Helmet_5 - 5 speed (m/s),DARKO_Robot - 1 speed (m/s),DARKO_Robot - 2 speed (m/s),DARKO_Robot - 3 speed (m/s),DARKO_Robot - 4 speed (m/s),DARKO_Robot - 5 speed (m/s),DARKO_Robot - 6 speed (m/s),DARKO_Robot - 7 speed (m/s)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.279205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.061406,0.070445,0.057956,0.023527,0.009953,0.016463,0.009536,0.002218,0.007603,0.004042
2,1.296423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.063915,0.060110,0.070143,0.011199,0.022765,0.009437,0.008659,0.649314,0.008514,0.167005
3,1.347425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.065363,0.059433,0.057183,0.004041,0.030755,0.002005,0.009668,0.118827,0.037569,0.053291
4,1.114409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.054536,0.074158,0.070767,0.012594,0.006569,0.004946,0.006037,0.019000,0.023880,0.011674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,0.295983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.103900,0.201603,0.045680,0.013357,0.002177,0.912211,0.005036,0.007291,0.013521,0.277358
163,0.410828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.102151,0.376194,0.148007,0.003151,0.007336,0.009155,0.006049,0.018877,0.019508,0.010187
164,0.265015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.093445,0.088821,0.295375,0.011308,0.009335,0.012436,0.007611,0.013048,0.014741,0.015853
165,0.517109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.081031,0.074364,0.059416,0.010685,0.028070,0.006662,0.003596,0.024183,0.019766,0.020277


In [23]:
features_df.shape

(167, 240)

In [24]:
features_df.index = features_df.Frame

In [25]:
features_cat = preprocessed_trajectories.join(features_df)

In [26]:
features_cat[features_cat.columns[features_cat.columns.str.contains("speed")]]

,Helmet_4 - 1 speed (m/s),Helmet_4 - 2 speed (m/s),Helmet_4 - 3 speed (m/s),Helmet_4 - 4 speed (m/s),Helmet_4 - 5 speed (m/s),LO1 - 1 speed (m/s),LO1 - 2 speed (m/s),LO1 - 3 speed (m/s),LO1 - 4 speed (m/s),LO1 - 5 speed (m/s),...,Helmet_5 - 3 speed (m/s),Helmet_5 - 4 speed (m/s),Helmet_5 - 5 speed (m/s),DARKO_Robot - 1 speed (m/s),DARKO_Robot - 2 speed (m/s),DARKO_Robot - 3 speed (m/s),DARKO_Robot - 4 speed (m/s),DARKO_Robot - 5 speed (m/s),DARKO_Robot - 6 speed (m/s),DARKO_Robot - 7 speed (m/s)
Frame,,,,,,,,,,,,,,,,,,,,,
15033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15034,1.279205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.061406,0.070445,0.057956,0.023527,0.009953,0.016463,0.009536,0.002218,0.007603,0.004042
15035,1.296423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.063915,0.060110,0.070143,0.011199,0.022765,0.009437,0.008659,0.649314,0.008514,0.167005
15036,1.347425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.065363,0.059433,0.057183,0.004041,0.030755,0.002005,0.009668,0.118827,0.037569,0.053291
15037,1.114409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.054536,0.074158,0.070767,0.012594,0.006569,0.004946,0.006037,0.019000,0.023880,0.011674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15195,0.295983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.103900,0.201603,0.045680,0.013357,0.002177,0.912211,0.005036,0.007291,0.013521,0.277358
15196,0.410828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.102151,0.376194,0.148007,0.003151,0.007336,0.009155,0.006049,0.018877,0.019508,0.010187
15197,0.265015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.093445,0.088821,0.295375,0.011308,0.009335,0.012436,0.007611,0.013048,0.014741,0.015853


In [26]:
rotations_cols = features_cat[
    features_cat.columns[features_cat.columns.str.contains(r"R(\d)")]
].columns.tolist()
rotations_cols

/tmp/ipykernel_49143/2502953472.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_cat.columns[features_cat.columns.str.contains(r"R(\d)")]


['DARKO_Robot R0',
 'DARKO_Robot R1',
 'DARKO_Robot R2',
 'DARKO_Robot R3',
 'DARKO_Robot R4',
 'DARKO_Robot R5',
 'DARKO_Robot R6',
 'DARKO_Robot R7',
 'DARKO_Robot R8',
 'Helmet_1 R0',
 'Helmet_1 R1',
 'Helmet_1 R2',
 'Helmet_1 R3',
 'Helmet_1 R4',
 'Helmet_1 R5',
 'Helmet_1 R6',
 'Helmet_1 R7',
 'Helmet_1 R8',
 'Helmet_10 R0',
 'Helmet_10 R1',
 'Helmet_10 R2',
 'Helmet_10 R3',
 'Helmet_10 R4',
 'Helmet_10 R5',
 'Helmet_10 R6',
 'Helmet_10 R7',
 'Helmet_10 R8',
 'Helmet_2 R0',
 'Helmet_2 R1',
 'Helmet_2 R2',
 'Helmet_2 R3',
 'Helmet_2 R4',
 'Helmet_2 R5',
 'Helmet_2 R6',
 'Helmet_2 R7',
 'Helmet_2 R8',
 'Helmet_3 R0',
 'Helmet_3 R1',
 'Helmet_3 R2',
 'Helmet_3 R3',
 'Helmet_3 R4',
 'Helmet_3 R5',
 'Helmet_3 R6',
 'Helmet_3 R7',
 'Helmet_3 R8',
 'Helmet_4 R0',
 'Helmet_4 R1',
 'Helmet_4 R2',
 'Helmet_4 R3',
 'Helmet_4 R4',
 'Helmet_4 R5',
 'Helmet_4 R6',
 'Helmet_4 R7',
 'Helmet_4 R8',
 'Helmet_5 R0',
 'Helmet_5 R1',
 'Helmet_5 R2',
 'Helmet_5 R3',
 'Helmet_5 R4',
 'Helmet_5 R5',
 'He

In [27]:
features_filtered = features_cat[
    ["Frame"] + 
    features_cat.columns[
        (features_cat.columns.str.endswith("X"))
        | (features_cat.columns.str.endswith("Y"))
        | (features_cat.columns.str.endswith("Z"))
        | (features_cat.columns.str.endswith("speed (m/s)"))
    ].tolist()
    + rotations_cols
]

In [28]:
features_filtered

,Frame,DARKO_Robot - 1 X,DARKO_Robot - 1 Y,DARKO_Robot - 1 Z,DARKO_Robot - 2 X,DARKO_Robot - 2 Y,DARKO_Robot - 2 Z,DARKO_Robot - 3 X,DARKO_Robot - 3 Y,DARKO_Robot - 3 Z,...,Helmet_9 R8,LO1 R0,LO1 R1,LO1 R2,LO1 R3,LO1 R4,LO1 R5,LO1 R6,LO1 R7,LO1 R8
0,15033,-3.886030,-1.188737,0.692199,-3.821085,-1.192664,0.312282,-4.646152,-0.815854,0.862110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15034,-3.885826,-1.188807,0.692293,-3.821118,-1.192571,0.312269,-4.646307,-0.815808,0.862079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15035,-3.885924,-1.188764,0.692260,-3.820947,-1.192614,0.312413,-4.646390,-0.815799,0.862035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15036,-3.885944,-1.188776,0.692293,-3.821163,-1.192532,0.312210,-4.646409,-0.815795,0.862040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15037,-3.885823,-1.188808,0.692279,-3.821138,-1.192589,0.312231,-4.646387,-0.815816,0.862001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,15195,-3.885874,-1.188801,0.692213,-3.821060,-1.192599,0.312257,-4.646297,-0.815801,0.862099,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,15196,-3.885843,-1.188797,0.692209,-3.821114,-1.192552,0.312273,-4.646381,-0.815766,0.862109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,15197,-3.885919,-1.188756,0.692282,-3.821062,-1.192629,0.312282,-4.646347,-0.815836,0.862012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,15198,-3.885917,-1.188834,0.692209,-3.821225,-1.192589,0.312057,-4.646404,-0.815806,0.861995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Visualization

In [29]:
import plotly.express as px
import plotly.graph_objects as go

In [30]:
features_filtered.columns[features_filtered.columns.str.contains("G3D")]

Index(['Helmet_5 TB2_G3D_X', 'Helmet_5 TB2_G3D_Y', 'Helmet_5 TB2_G3D_Z',
       'Helmet_6 TB3_G3D_X', 'Helmet_6 TB3_G3D_Y', 'Helmet_6 TB3_G3D_Z'],
      dtype='object')

In [31]:
HELMET_TO_VISUALIZE = "Helmet_5"

In [32]:
gaze_data = features_filtered[
    [
        f"{HELMET_TO_VISUALIZE} TB2_G3D_X",
        f"{HELMET_TO_VISUALIZE} TB2_G3D_Y",
        f"{HELMET_TO_VISUALIZE} TB2_G3D_Z",
    ]
].dropna()
gaze_data.shape

(166, 3)

In [33]:
centroids_data = features_filtered[features_filtered.columns[features_filtered.columns.str.contains(f"{HELMET_TO_VISUALIZE} Centroid")]].dropna()
centroids_data.shape

(167, 3)

In [34]:
centroids_data

,Helmet_5 Centroid_X,Helmet_5 Centroid_Y,Helmet_5 Centroid_Z
0,108.96330,3465.82242,1536.80798
1,109.38860,3466.07652,1536.40857
2,109.79436,3466.34948,1536.00378
3,110.18250,3466.63918,1535.64289
4,110.61231,3466.93717,1535.25926
...,...,...,...
162,136.24555,3461.02575,1523.77589
163,136.87356,3459.51405,1523.65217
164,137.27808,3459.16355,1523.96066
165,137.47919,3458.42031,1523.75478


In [35]:
lo_info = GroupsInfo(
    element_id="LO1", markers_pattern_re=r"LO1 - (\d).*", label_sep=" - "
)
darko_info = GroupsInfo(
    element_id="DARKO", markers_pattern_re=r"DARKO_Robot - (\d).*", label_sep=" - "
)
helmets_info = GroupsInfo(
    element_id="Helmet", markers_pattern_re=r"Helmet_(\d+ - \d).*", label_sep="_"
)
tobii_info = GroupsInfo(
    element_id="Helmet", markers_pattern_re=r"Helmet_(\d+ TB\d)_G3D.*", label_sep="_"
)
centroids_info = GroupsInfo(
        element_id="Helmet",
        markers_pattern_re=r"Helmet_(\d+) Centroid_.*",
        label_sep="_",
    )
rotations_info = GroupsInfo(
        element_id="Helmet",
        markers_pattern_re=r"Helmet_(\d+) R.*",
        label_sep="_",
    )


In [68]:
def transform_df2plotly(
    input_df: pd.DataFrame, groups_info
) -> pd.DataFrame:
    """Transform a dataframe into the plotly best suited format
    |   Frame    |   X (m)  |   Y (m)  |   eid   |   mid   |

    being `eid` the element identifier (e.g. Helmet, DARKO, etc), and `mid` the marker identifier

    Parameters
    ----------
    input_df
        input pandas DataFrame
    element_id
        see eid explanation above
    markers_pattern_re
        regex to groupby element id and markers
    sep
        separation used in col name form element id and marker id

    Returns
    -------
        Transformed pandas DataFrame
    """
    groups_info = [groups_info] if isinstance(groups_info, GroupsInfo) else groups_info
    groups = []
    for group_info in groups_info:
        element_id = group_info.element_id
        elements_grouped = input_df.groupby(
            input_df.columns.str.extract(group_info.markers_pattern_re, expand=False),
            axis=1,
        )
        for group_name, group in elements_grouped:
            if element_id == "Helmet" and len(group_name.split("-")) == 1:
                tobii = len(group_name.split(" ")) == 2
                # eyt or centroids
                _mapping_cols = (
                    get_mapping_cols_tobii(
                        element_id,
                        group_name,
                        group_info.label_sep,
                    )
                    if tobii
                    else get_mapping_cols_centroids(
                        element_id, group_name, group_info.label_sep
                    )
                )
                group = group.rename(_mapping_cols, axis=1)
                eid = element_id + "_" + group_name.split(" ")[0]
            else:
                _mapping_cols = get_mapping_cols(
                    element_id, group_name, group_info.label_sep
                )
                group = group.rename(_mapping_cols, axis=1)
                eid = (
                    element_id + "_" + group_name.split(" - ")[0]
                    if element_id == "Helmet"
                    else element_id
                )
                mid = (
                    group_name.split(" - ")[1] if element_id == "Helmet" else group_name
                )
                group["mid"] = mid
            group["eid"] = eid
            groups.append(group)
    out_df = pd.concat(groups, axis=0)
    return out_df

In [69]:
transformed = transform_df2plotly(
        input_df=features_filtered.copy(),
        groups_info=[tobii_info],
    )

In [70]:
transformed

,TB_G3D X (m),TB_G3D Y (m),TB_G3D Z (m),eid
0,NaN,NaN,NaN,Helmet_5
1,387.696357,3356.778699,1768.926842,Helmet_5
2,389.758957,3356.454384,1771.104233,Helmet_5
3,390.053442,3356.764545,1770.859035,Helmet_5
4,390.347665,3357.805761,1771.083621,Helmet_5
...,...,...,...,...
162,NaN,NaN,NaN,Helmet_6
163,NaN,NaN,NaN,Helmet_6
164,NaN,NaN,NaN,Helmet_6
165,NaN,NaN,NaN,Helmet_6


In [71]:
transformed_cent = transform_df2plotly(
        input_df=features_filtered.copy(),
        groups_info=[centroids_info],
    )

In [72]:
transformed_cent = transformed_cent.sort_index()

In [73]:
transformed_cent

,Centroid X (m),Centroid Y (m),Centroid Z (m),eid
0,NaN,NaN,NaN,Helmet_1
0,NaN,NaN,NaN,Helmet_3
0,108.96330,3465.82242,1536.80798,Helmet_5
0,NaN,NaN,NaN,Helmet_6
0,NaN,NaN,NaN,Helmet_2
...,...,...,...,...
166,-3430.41278,739.85130,1861.86219,Helmet_2
166,3005.17148,-676.49252,1692.34251,Helmet_10
166,NaN,NaN,NaN,Helmet_1
166,-3290.43134,1811.84756,1826.68101,Helmet_8


In [74]:
transformed_rotations = transform_df2plotly(
        input_df=features_filtered.copy(),
        groups_info=[rotations_info],
    )

In [75]:
transformed_rotations = transformed_rotations.sort_index()

In [76]:
transformed_rotations

,R0,R1,R2,R3,R4,R5,R6,R7,R8,eid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_3
0,0.42649,0.19841,-0.88246,-0.42769,0.90392,-0.00346,0.79699,0.37890,0.47037,Helmet_5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_2
...,...,...,...,...,...,...,...,...,...,...
166,0.81265,0.52180,0.25948,-0.55166,0.83232,0.05396,-0.18782,-0.18700,0.96424,Helmet_2
166,0.97085,0.13625,0.19717,-0.13458,0.99066,-0.02192,-0.19832,-0.00525,0.98012,Helmet_10
166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_1
166,0.90196,0.39688,0.17014,-0.40798,0.91233,0.03466,-0.14147,-0.10068,0.98481,Helmet_8


In [77]:
(transformed_rotations.eid.values == transformed_cent.eid.values).sum() == len(transformed_rotations)

True

In [78]:
cent_rotations = pd.concat([transformed_cent, transformed_rotations.drop("eid", axis = 1)], axis = 1)

In [79]:
cent_rotations

,Centroid X (m),Centroid Y (m),Centroid Z (m),eid,R0,R1,R2,R3,R4,R5,R6,R7,R8
0,NaN,NaN,NaN,Helmet_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,Helmet_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,108.96330,3465.82242,1536.80798,Helmet_5,0.42649,0.19841,-0.88246,-0.42769,0.90392,-0.00346,0.79699,0.37890,0.47037
0,NaN,NaN,NaN,Helmet_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,Helmet_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,-3430.41278,739.85130,1861.86219,Helmet_2,0.81265,0.52180,0.25948,-0.55166,0.83232,0.05396,-0.18782,-0.18700,0.96424
166,3005.17148,-676.49252,1692.34251,Helmet_10,0.97085,0.13625,0.19717,-0.13458,0.99066,-0.02192,-0.19832,-0.00525,0.98012
166,NaN,NaN,NaN,Helmet_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,-3290.43134,1811.84756,1826.68101,Helmet_8,0.90196,0.39688,0.17014,-0.40798,0.91233,0.03466,-0.14147,-0.10068,0.98481


In [87]:
transformed_trajectories = transform_df2plotly(
        input_df=features_filtered.copy(),
        groups_info=[helmets_info],
    )
transformed_trajectories = transformed_trajectories.sort_index()

In [89]:
transformed_trajectories

,X (m),Y (m),Z (m),speed (m/s),mid,eid
0,NaN,NaN,NaN,NaN,1,Helmet_1
0,0.298228,-0.408581,1.708787,NaN,1,Helmet_10
0,-4.784008,0.523736,1.788615,NaN,3,Helmet_8
0,NaN,NaN,NaN,NaN,3,Helmet_4
0,NaN,NaN,NaN,NaN,4,Helmet_2
...,...,...,...,...,...,...
166,-3.269827,1.881261,1.857702,1.123939,4,Helmet_8
166,-3.385920,0.837854,1.851159,1.174227,1,Helmet_2
166,NaN,NaN,NaN,0.000000,5,Helmet_4
166,0.113074,3.511985,1.477777,0.079656,4,Helmet_5


In [90]:
transformed.sort_index()

,TB_G3D X (m),TB_G3D Y (m),TB_G3D Z (m),eid
0,NaN,NaN,NaN,Helmet_5
0,NaN,NaN,NaN,Helmet_6
1,NaN,NaN,NaN,Helmet_6
1,387.696357,3356.778699,1768.926842,Helmet_5
2,389.758957,3356.454384,1771.104233,Helmet_5
...,...,...,...,...
164,411.174035,3367.010134,1798.774197,Helmet_5
165,NaN,NaN,NaN,Helmet_6
165,411.066621,3366.247297,1798.866850,Helmet_5
166,413.120615,3365.938535,1800.686487,Helmet_5


In [91]:
def filter_best_markers(elements_cat_df: pd.DataFrame, nan_counter_by_marker):
    elements_filtered_by_best_marker = []
    for instance_id, nans_counter in nan_counter_by_marker.items():
        best_marker_id = min(
            nans_counter,
            key=nans_counter.get,
        )
        elements_filtered_by_best_marker.append(
            elements_cat_df[
                (elements_cat_df.eid == instance_id)
                & (elements_cat_df.mid == best_marker_id)
            ]
        )
    out_df = pd.concat(elements_filtered_by_best_marker, axis=0)
    out_df = out_df.sort_index().reset_index()
    return out_df

In [92]:
best_makers_df = filter_best_markers(
        elements_cat_df=transformed_trajectories.copy(), nan_counter_by_marker=best_markers
    )

In [93]:
best_makers_df.columns

Index(['index', 'X (m)', 'Y (m)', 'Z (m)', 'speed (m/s)', 'mid', 'eid'], dtype='object')

In [94]:
ORIGIN = (0,0,0)
X = (1, 0, 0)
Y = (0, 1, 0)
Z = (0, 0, 1)
COLOR1 = 'red'
COLOR2 = 'green'
COLOR3 = 'blue'
COLOR4 = 'orange'

In [95]:
best_makers_df["speed (m/s)"]

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
1164    1.174227
1165    1.025386
1166    0.073633
1167    0.853347
1168    1.648100
Name: speed (m/s), Length: 1169, dtype: float64

<TB2_G3D_X, TB2_G3D_Y, TB2_G3D_Z> is the transformation from the helmet frame to a convergence point
of the eye gaze.

In [73]:
# fig = px.scatter_3d(
#     transformed[:40000],
#     x="TB_G3D X (m)",
#     y="TB_G3D Y (m)",
#     z="TB_G3D Z (m)",
#     color="eid"
# )
fig = go.Figure([])
fig.add_trace(
    go.Scatter3d(
        x=[ORIGIN[0], X[0], None],
        y=[ORIGIN[1], X[1], None],
        z=[ORIGIN[2], X[2], None],
        mode="lines",
        line=dict(color="red", width=5),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scatter3d(
        x=[ORIGIN[0], Y[0], None],
        y=[ORIGIN[1], Y[1], None],
        z=[ORIGIN[2], Y[2], None],
        mode="lines",
        line=dict(color="green", width=5),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scatter3d(
        x=[ORIGIN[0], Z[0], None],
        y=[ORIGIN[1], Z[1], None],
        z=[ORIGIN[2], Z[2], None],
        mode="lines",
        line=dict(color="blue", width=5),
        showlegend=False,
    )
)

fig.add_cone(
    x=[1],
    y=[0],
    z=[0],
    u=[1],
    v=[0],
    w=[0],
    colorscale=[[0, COLOR1], [1, COLOR1]],
    showscale=False,
)

fig.add_cone(
    x=[0],
    y=[1],
    z=[0],
    u=[0],
    v=[1],
    w=[0],
    colorscale=[[0, COLOR2], [1, COLOR2]],
    showscale=False,
)

fig.add_cone(
    x=[0],
    y=[0],
    z=[Z[2]],
    u=[0],
    v=[0],
    w=[Z[2]],
    colorscale=[[0, COLOR3], [1, COLOR3]],
    showscale=False,
)